<a href="https://colab.research.google.com/github/NREL/boptest-service/blob/develop/docs/Introduction_to_BOPTEST_Service_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BOPTEST Service APIs

This is an introduction to the BOPTEST-Service APIs.


In [17]:
import requests
import json

BOPTEST_URL = 'https://api.boptest.net'

def pretty_print(json_message):
    print(json.dumps(json_message, indent=2))

## List the available test cases


In [18]:
all_testcases = requests.get(f"{BOPTEST_URL}/testcases")
pretty_print(all_testcases.json()[:8]) # Printing out the first eight testcases

[
  {
    "testcaseid": "bestest_air"
  },
  {
    "testcaseid": "bestest_hydronic"
  },
  {
    "testcaseid": "bestest_hydronic_heat_pump"
  },
  {
    "testcaseid": "multizone_residential_hydronic"
  },
  {
    "testcaseid": "singlezone_commercial_hydronic"
  },
  {
    "testcaseid": "testcase1"
  },
  {
    "testcaseid": "testcase2"
  },
  {
    "testcaseid": "testcase3"
  }
]


## Select a test case

The POST `testcases/{testcase_name}/select` API is used to select a test case. A unique `test_id` is returned and used as a reference to the test in subsequent API calls.


In [19]:
testcase_name = "bestest_hydronic_heat_pump"

test_id = requests.post(f"{BOPTEST_URL}/testcases/{testcase_name}/select").json()['testid']
print(f"The test id for this particular test is {test_id}.")

The test id for this particular test is abfa0efa-3096-4062-8b04-c8268e0b08f8.


## Select the test scenario

Scenarios are pre-defined time periods and conditions for each test. Setting a scenario will initialize the test to start at the beginning of the pre-defined time period. The available scenarios for each BOPTEST test case is provided in the [BOPTEST documentation](https://ibpsa.github.io/project1-boptest/testcases/index.html). For example, the "bestest_hydronic_heat_pump" we are using in this tutorial has the following time periods and conditions available for scenario.  

Time periods:
1. peak_heat_day
2. typical_heat_day

Electricity prices:
1. constant
2. dynamic
3. highly_dynamic

The PUT `/scenario` API is used to select a test scenario.

In [20]:
# Initialize using /scenario
y = requests.put(f'{BOPTEST_URL}/scenario/{test_id}', 
                 data={'time_period':'peak_heat_day',
                       'electricity_price':'constant'}).json()['payload']
pretty_print(y)

{
  "electricity_price": "constant",
  "time_period": {
    "ovePum_activate": 0,
    "weaSta_reaWeaNOpa_y": 0.2,
    "reaPFan_y": 510.20408163265313,
    "oveTSet_u": 294.34999999999997,
    "reaQHeaPumCon_y": 7386.771485507761,
    "reaTRet_y": 302.40029127477544,
    "weaSta_reaWeaPAtm_y": 101325,
    "weaSta_reaWeaTBlaSky_y": 260.82015225846817,
    "reaQHeaPumEva_y": -5001.493945256057,
    "weaSta_reaWeaNTot_y": 0.2,
    "weaSta_reaWeaSolAlt_y": -1.0200406146124426,
    "reaTZon_y": 294.4643919860063,
    "weaSta_reaWeaHHorIR_y": 262,
    "weaSta_reaWeaSolTim_y": 1379289.1064134557,
    "oveHeaPumY_u": 0.6283099681151593,
    "weaSta_reaWeaCloTim_y": 1382400,
    "oveHeaPumY_activate": 0,
    "reaPPumEmi_y": 20.498644281163177,
    "weaSta_reaWeaHGloHor_y": 0,
    "weaSta_reaWeaHDifHor_y": 0,
    "oveTSet_activate": 0,
    "weaSta_reaWeaRelHum_y": 0.9,
    "reaTSetHea_y": 294.15,
    "reaCO2RooAir_y": 754.1571682156247,
    "weaSta_reaWeaSolDec_y": -0.36698636938254725,
    "oveP

## Test measurements

Measurements refer to the outputs from the test case model. Measurements are like properties that define the model state. Each test case has its specific set of measurements. You can check what measurements are available for a test with the GET `/measurements` request. 

The API response will tell you what are 
1. The minimum values for the measurement.
2. A short description of the measurement, 
3. The measurement unit.
4. The maximum value for the measurement. 

In [21]:
measurements = requests.get(f'{BOPTEST_URL}/measurements/{test_id}').json()['payload']
pretty_print(measurements)

{
  "weaSta_reaWeaPAtm_y": {
    "Minimum": null,
    "Description": "Atmospheric pressure measurement",
    "Unit": "Pa",
    "Maximum": null
  },
  "reaPFan_y": {
    "Minimum": null,
    "Description": "Electrical power of the heat pump evaporator fan",
    "Unit": "W",
    "Maximum": null
  },
  "reaQHeaPumCon_y": {
    "Minimum": null,
    "Description": "Heat pump thermal power exchanged in the condenser",
    "Unit": "W",
    "Maximum": null
  },
  "reaTRet_y": {
    "Minimum": null,
    "Description": "Return water temperature from radiant floor",
    "Unit": "K",
    "Maximum": null
  },
  "weaSta_reaWeaNOpa_y": {
    "Minimum": null,
    "Description": "Opaque sky cover measurement",
    "Unit": "1",
    "Maximum": null
  },
  "weaSta_reaWeaTBlaSky_y": {
    "Minimum": null,
    "Description": "Black-body sky temperature measurement",
    "Unit": "K",
    "Maximum": null
  },
  "reaQHeaPumEva_y": {
    "Minimum": null,
    "Description": "Heat pump thermal power exchanged in 

## Advance the test

A test is moved forward in time by calling the `/advance` API.

The response payload of the `/advance` API will return the values of the current measurements.

In [22]:
# A basic advance step
y = requests.post(f'{BOPTEST_URL}/advance/{test_id}').json()['payload']

pretty_print(y)

{
  "ovePum_activate": 0,
  "weaSta_reaWeaNOpa_y": 0.6000000000000001,
  "reaPFan_y": 510.20408163265313,
  "oveTSet_u": 294.34999999999997,
  "reaQHeaPumCon_y": 6496.430936371346,
  "reaTRet_y": 302.1780288330197,
  "weaSta_reaWeaPAtm_y": 101325,
  "weaSta_reaWeaTBlaSky_y": 263.1259141058702,
  "reaQHeaPumEva_y": -4322.109373135557,
  "weaSta_reaWeaNTot_y": 0.6000000000000001,
  "weaSta_reaWeaSolAlt_y": -1.0485334341454244,
  "reaTZon_y": 294.55639797609035,
  "weaSta_reaWeaHHorIR_y": 272,
  "weaSta_reaWeaSolTim_y": 1382888.2797402868,
  "oveHeaPumY_u": 0.5280734825493557,
  "weaSta_reaWeaCloTim_y": 1386000,
  "oveHeaPumY_activate": 0,
  "reaPPumEmi_y": 20.498644281163177,
  "weaSta_reaWeaHGloHor_y": 0,
  "weaSta_reaWeaHDifHor_y": 0,
  "oveTSet_activate": 0,
  "weaSta_reaWeaRelHum_y": 0.89,
  "reaTSetHea_y": 294.15,
  "reaCO2RooAir_y": 775.9300128209737,
  "weaSta_reaWeaSolDec_y": -0.36685410880578145,
  "ovePum_u": 1,
  "reaPHeaPum_y": 2174.321563211399,
  "weaSta_reaWeaHDirNor_y": 0

## Test inputs

The GET `/inputs` API will return the input variables for the selected test case.

The API response will tell you what are 
1. The minimum values for the variable.
2. A short description of the variable, 
3. The unit for the variable.
4. The maximum value for the variable. 

 

In [23]:
# Get inputs available
inputs = requests.get(f'{BOPTEST_URL}/inputs/{test_id}').json()['payload']

pretty_print(inputs)

{
  "oveTSet_activate": {
    "Minimum": null,
    "Description": "Activation for Zone operative temperature setpoint",
    "Unit": null,
    "Maximum": null
  },
  "ovePum_activate": {
    "Minimum": null,
    "Description": "Activation for Integer signal to control the emission circuit pump either on or off",
    "Unit": null,
    "Maximum": null
  },
  "ovePum_u": {
    "Minimum": 0,
    "Description": "Integer signal to control the emission circuit pump either on or off",
    "Unit": "1",
    "Maximum": 1
  },
  "oveHeaPumY_u": {
    "Minimum": 0,
    "Description": "Heat pump modulating signal for compressor speed between 0 (not working) and 1 (working at maximum capacity)",
    "Unit": "1",
    "Maximum": 1
  },
  "oveTSet_u": {
    "Minimum": 278.15,
    "Description": "Zone operative temperature setpoint",
    "Unit": "K",
    "Maximum": 308.15
  },
  "oveHeaPumY_activate": {
    "Minimum": null,
    "Description": "Activation for Heat pump modulating signal for compressor spee

## Set input values and advance


The input variables can be set to desired values in a python dictionary, which is passed as an argument to the `/advance` API.

In [25]:
# Example input dictionary
u = {'oveHeaPumY_u':0.5,
     'oveHeaPumY_activate': 1}

# advance with u
y = requests.post(f'{BOPTEST_URL}/advance/{test_id}', data=u).json()['payload']

pretty_print(y)

{
  "ovePum_activate": 0,
  "weaSta_reaWeaNOpa_y": 0,
  "reaPFan_y": 510.20408163265313,
  "oveTSet_u": 294.34999999999997,
  "reaQHeaPumCon_y": 6197.232442685376,
  "reaTRet_y": 301.6817774604096,
  "weaSta_reaWeaPAtm_y": 101325,
  "weaSta_reaWeaTBlaSky_y": 258.3367101474463,
  "reaQHeaPumEva_y": -4093.7235704880295,
  "weaSta_reaWeaNTot_y": 0,
  "weaSta_reaWeaSolAlt_y": -0.8903885648155656,
  "reaTZon_y": 294.5382983159215,
  "weaSta_reaWeaHHorIR_y": 253,
  "weaSta_reaWeaSolTim_y": 1390086.6295405712,
  "oveHeaPumY_u": 0.5,
  "weaSta_reaWeaCloTim_y": 1393200,
  "oveHeaPumY_activate": 1,
  "reaPPumEmi_y": 20.498644281163177,
  "weaSta_reaWeaHGloHor_y": 0,
  "weaSta_reaWeaHDifHor_y": 0,
  "oveTSet_activate": 0,
  "weaSta_reaWeaRelHum_y": 0.86,
  "reaTSetHea_y": 294.15,
  "reaCO2RooAir_y": 797.1465635321247,
  "weaSta_reaWeaSolDec_y": -0.36658901571136265,
  "ovePum_u": 1,
  "reaPHeaPum_y": 2103.508872139368,
  "weaSta_reaWeaHDirNor_y": 0,
  "reaTSetCoo_y": 297.15,
  "weaSta_reaWeaWinDi

## Stop the test

The PUT `/stop` request will stop the test case as shown below. From this point forward the `test_id` will be invalid.

In [26]:
stop_response = requests.put(f"{BOPTEST_URL}/stop/{test_id}")
if stop_response.status_code == 200:
  print("Successfully stopped the test!")
else:
  print(f"Couldn't stop test with status code: {stop_response.status_code}!")


Successfully stopped the test!
